In [1]:
import pandas as pd
import numpy as np
import cv2
import math
import os
import glob

In [2]:
#directory where behavior videos are located
video_directory = '01_Raw_videos'
dataframe_directory = '06_Unified_dataframe'
output_directory = '07_Processed_videos'
file_location = os.path.join(video_directory, '*.mp4')

# create list of filenames for every video to be processed
# this list is the input for the video processing function
filenames = glob.glob(file_location)
videoname = []
for f in filenames:
    # establish name for output file from the input filename
    x = os.path.split(f)
    x = x[1].split('.mp4')
    x = x[0]
    videoname.append(x)
videoname

['Gal_35_father_retreive_trial_1']

In [3]:
hdf_file = os.path.join(dataframe_directory, x)
hdf_var = glob.glob(hdf_file + '*')
Full = pd.read_hdf(hdf_var[0])

In [4]:
Full['Infant1']

bodyparts   head                   middle_head                    middle_tail  \
coords         x      y likelihood           x       y likelihood           x   
0            NaN    NaN        NaN         NaN     NaN        NaN         NaN   
1            NaN    NaN        NaN         NaN     NaN        NaN         NaN   
2            NaN    NaN        NaN         NaN     NaN        NaN         NaN   
3            NaN    NaN        NaN         NaN     NaN        NaN         NaN   
4            NaN    NaN        NaN         NaN     NaN        NaN         NaN   
...          ...    ...        ...         ...     ...        ...         ...   
11646      304.0  380.5   0.010002       286.0  351.25   0.010002       287.0   
11647      304.0  380.5   0.010002       286.0  351.25   0.010002       287.0   
11648      304.0  380.5   0.010002       286.0  351.25   0.010002       287.0   
11649      304.0  380.5   0.010002       286.0  351.25   0.010002       287.0   
11650      304.0  380.5   0.010002       286.0  351.25   0.010002       287.0   

bodyparts                      tail                    
coords         y likelihood       x      y likelihood  
0            NaN        NaN     NaN    NaN        NaN  
1            NaN        NaN     NaN    NaN        NaN  
2            NaN        NaN     NaN    NaN        NaN  
3            NaN        NaN     NaN    NaN        NaN  
4            NaN        NaN     NaN    NaN        NaN  
...          ...        ...     ...    ...        ...  
11646      337.0   0.010002  284.25  330.0   0.010002  
11647      337.0   0.010002  284.25  330.0   0.010002  
11648      337.0   0.010002  284.25  330.0   0.010002  
11649      337.0   0.010002  284.25  330.0   0.010002  
11650      337.0   0.010002  284.25  330.0   0.010002  

[11651 rows x 12 columns]

# Create video with bodyparts/nest overlay

In [5]:
for x in videoname:
    # Video capture
    video_file = os.path.join(video_directory, x)
    video_var = glob.glob(video_file + '.mp4')
    vcapture = cv2.VideoCapture(video_var[0])
    width = int(vcapture.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(vcapture.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = vcapture.get(cv2.CAP_PROP_FPS)

    # Define codec and create video writer
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out_file = os.path.join(output_directory, x)
    out = cv2.VideoWriter(out_file + '_processed.avi', fourcc, fps, (width, height))
    
    
    hdf_file = os.path.join(dataframe_directory, x)
    hdf_var = glob.glob(hdf_file + '*')
    Full = pd.read_hdf(hdf_var[0])
    
    # Create bodyparts list for plotting
    Infant_bodyparts = Full['Infant1'].columns.get_level_values(level=0)
    Infant_bodyparts = set(Infant_bodyparts.tolist())
    
    Adult_bodyparts = Full['Adult'].columns.get_level_values(level=0)
    Adult_bodyparts = set(Adult_bodyparts.tolist())

    count = 0
    success = True
    while success:
        print("frame: ", count)
        # Read next image
        success, image = vcapture.read()
        if success:
            # OpenCV returns images as BGR, convert to RGB
            #image = image[..., ::-1]
        
            ##### THIS IS THE IMPORTANT PART ######
            df_polygon_coordinates = Full.iloc[count]
            overlay = image.copy()
            output = image.copy()
        
            ##############################################
            ###############    Nest   ####################
            ##############################################
            
            # Set the transparency value for the nest overlay
            nest_alpha = 0.2
        
            # Subset coordinates belonging to the nest
            df_nest = df_polygon_coordinates.filter(regex='nest')
    
            # Count how many nest objects are present in the image
            N = df_nest.size
        
            # Create empty dict to store coordinates
            shapes = {}
        
            # Populate dictionary containing arrays of individual nest objects
            for i in range(N):
                nest_test = df_nest.iloc[i]
                if np.isnan(nest_test).any():
                    continue
                shapes.update({"nest_" + str(i):np.array(nest_test, np.int32)})
        
            # Create polygon for each array
            for x in shapes.values():
                cv2.fillPoly(overlay, [x], (255, 255, 0))
                
            polygon_image = cv2.addWeighted(overlay, nest_alpha, image, 1-nest_alpha, 0)
            
        
            ##############################################
            ###########   Adult Bodyparts   ##############
            ##############################################

            cutoff = 0.9

            # Subset coordinates belonging to the Adult bodyparts
            df_adult = df_polygon_coordinates['Adult']

            # Create empty dictionaries to store coordinates
            keypoints = {}

            for part in Adult_bodyparts:
                keypoints.update({part:list(df_adult[part])})
    
            # Create circle for each array
            for x in keypoints.values():
                pts3 = x[0:3]
                NaN = math.isnan(pts3[0])
                if NaN is True:
                    pts3[0] = 0
                    pts3[1] = 0
                    pts3[2] = 0
                pts3 = pts3[0:2]
                ptsX = tuple(list(map(int, pts3)))
    
                if x[2] > cutoff:
                    overlay = cv2.circle(polygon_image, ptsX, 5, (255, 0, 255), -1)
                    alpha = 1
                    output = cv2.addWeighted(overlay, alpha, polygon_image, 1-alpha, 0)
        
            ##############################################
            ###########  Infant_1 Bodyparts  #############
            ##############################################

            cutoff = 0.1
        
            # Subset coordinates belonging to the Infant1 bodyparts
            df_infant1 = df_polygon_coordinates['Infant1']

            keypoints = {}

            for part in Infant_bodyparts:
                keypoints.update({part:list(df_infant1[part])})

            # Create circle for each array
            for x in keypoints.values():
                pts3 = x[0:3]
                NaN = math.isnan(pts3[0])
                if NaN is True:
                    pts3[0] = 0
                    pts3[1] = 0
                    pts3[2] = 0
                NaN = math.isnan(pts3[1])
                if NaN is True:
                    pts3[0] = 0
                    pts3[1] = 0
                    pts3[2] = 0
                pts3 = pts3[0:2]
                ptsX = tuple(list(map(int, pts3)))
    
                if x[2] > cutoff:
                    overlay = cv2.circle(output, ptsX, 5, (0, 255, 0), -1)
                    alpha = 1
                    output = cv2.addWeighted(overlay, alpha, output, 1-alpha, 0)
                
            ##############################################
            ###########  Infant_2 Bodyparts  #############
            ##############################################
        
            cutoff = 0.1
        
            # Subset coordinates belonging to the Infant1 bodyparts
            df_infant2 = df_polygon_coordinates['Infant2']

            keypoints = {}

            for part in Infant_bodyparts:
                keypoints.update({part:list(df_infant2[part])})

            # Create circle for each array
            for x in keypoints.values():
                pts3 = x[0:3]
                NaN = math.isnan(pts3[0])
                if NaN is True:
                    pts3[0] = 0
                    pts3[1] = 0
                    pts3[2] = 0
                NaN = math.isnan(pts3[1])
                if NaN is True:
                    pts3[0] = 0
                    pts3[1] = 0
                    pts3[2] = 0
                pts3 = pts3[0:2]
                ptsX = tuple(list(map(int, pts3)))
    
                if x[2] > cutoff:
                    overlay = cv2.circle(output, ptsX, 5, (0, 0, 255), -1)
                    alpha = 1
                    output = cv2.addWeighted(output, alpha, image, 1-alpha, 0)

            # Write the altered frame
            out.write(output)

            count += 1
    vcapture.release()

frame:  0


<ipython-input-5-87a608078b80>:133: RuntimeWarning: invalid value encountered in greater
  if x[2] > cutoff:
<ipython-input-5-87a608078b80>:168: RuntimeWarning: invalid value encountered in greater
  if x[2] > cutoff:


frame:  1
frame:  2
frame:  3
frame:  4
frame:  5
frame:  6
frame:  7
frame:  8
frame:  9
frame:  10
frame:  11
frame:  12
frame:  13
frame:  14
frame:  15
frame:  16
frame:  17
frame:  18
frame:  19
frame:  20
frame:  21
frame:  22
frame:  23
frame:  24
frame:  25
frame:  26
frame:  27
frame:  28
frame:  29
frame:  30
frame:  31
frame:  32
frame:  33
frame:  34
frame:  35
frame:  36
frame:  37
frame:  38
frame:  39
frame:  40
frame:  41
frame:  42
frame:  43
frame:  44
frame:  45
frame:  46
frame:  47
frame:  48
frame:  49
frame:  50
frame:  51
frame:  52
frame:  53
frame:  54
frame:  55
frame:  56
frame:  57
frame:  58
frame:  59
frame:  60
frame:  61
frame:  62
frame:  63
frame:  64
frame:  65
frame:  66
frame:  67
frame:  68
frame:  69
frame:  70
frame:  71
frame:  72
frame:  73
frame:  74
frame:  75
frame:  76
frame:  77
frame:  78
frame:  79
frame:  80
frame:  81
frame:  82
frame:  83
frame:  84
frame:  85
frame:  86
frame:  87
frame:  88
frame:  89
frame:  90
frame:  91
frame:  

frame:  694
frame:  695
frame:  696
frame:  697
frame:  698
frame:  699
frame:  700
frame:  701
frame:  702
frame:  703
frame:  704
frame:  705
frame:  706
frame:  707
frame:  708
frame:  709
frame:  710
frame:  711
frame:  712
frame:  713
frame:  714
frame:  715
frame:  716
frame:  717
frame:  718
frame:  719
frame:  720
frame:  721
frame:  722
frame:  723
frame:  724
frame:  725
frame:  726
frame:  727
frame:  728
frame:  729
frame:  730
frame:  731
frame:  732
frame:  733
frame:  734
frame:  735
frame:  736
frame:  737
frame:  738
frame:  739
frame:  740
frame:  741
frame:  742
frame:  743
frame:  744
frame:  745
frame:  746
frame:  747
frame:  748
frame:  749
frame:  750
frame:  751
frame:  752
frame:  753
frame:  754
frame:  755
frame:  756
frame:  757
frame:  758
frame:  759
frame:  760
frame:  761
frame:  762
frame:  763
frame:  764
frame:  765
frame:  766
frame:  767
frame:  768
frame:  769
frame:  770
frame:  771
frame:  772
frame:  773
frame:  774
frame:  775
frame:  776
fram

frame:  1351
frame:  1352
frame:  1353
frame:  1354
frame:  1355
frame:  1356
frame:  1357
frame:  1358
frame:  1359
frame:  1360
frame:  1361
frame:  1362
frame:  1363
frame:  1364
frame:  1365
frame:  1366
frame:  1367
frame:  1368
frame:  1369
frame:  1370
frame:  1371
frame:  1372
frame:  1373
frame:  1374
frame:  1375
frame:  1376
frame:  1377
frame:  1378
frame:  1379
frame:  1380
frame:  1381
frame:  1382
frame:  1383
frame:  1384
frame:  1385
frame:  1386
frame:  1387
frame:  1388
frame:  1389
frame:  1390
frame:  1391
frame:  1392
frame:  1393
frame:  1394
frame:  1395
frame:  1396
frame:  1397
frame:  1398
frame:  1399
frame:  1400
frame:  1401
frame:  1402
frame:  1403
frame:  1404
frame:  1405
frame:  1406
frame:  1407
frame:  1408
frame:  1409
frame:  1410
frame:  1411
frame:  1412
frame:  1413
frame:  1414
frame:  1415
frame:  1416
frame:  1417
frame:  1418
frame:  1419
frame:  1420
frame:  1421
frame:  1422
frame:  1423
frame:  1424
frame:  1425
frame:  1426
frame:  1427

...

frame:  8950
frame:  8951
frame:  8952
frame:  8953
frame:  8954
frame:  8955
frame:  8956
frame:  8957
frame:  8958
frame:  8959
frame:  8960
frame:  8961
frame:  8962
frame:  8963
frame:  8964
frame:  8965
frame:  8966
frame:  8967
frame:  8968
frame:  8969
frame:  8970
frame:  8971
frame:  8972
frame:  8973
frame:  8974
frame:  8975
frame:  8976
frame:  8977
frame:  8978
frame:  8979
frame:  8980
frame:  8981
frame:  8982
frame:  8983
frame:  8984
frame:  8985
frame:  8986
frame:  8987
frame:  8988
frame:  8989
frame:  8990
frame:  8991
frame:  8992
frame:  8993
frame:  8994
frame:  8995
frame:  8996
frame:  8997
frame:  8998
frame:  8999
frame:  9000
frame:  9001
frame:  9002
frame:  9003
frame:  9004
frame:  9005
frame:  9006
frame:  9007
frame:  9008
frame:  9009
frame:  9010
frame:  9011
frame:  9012
frame:  9013
frame:  9014
frame:  9015
frame:  9016
frame:  9017
frame:  9018
frame:  9019
frame:  9020
frame:  9021
frame:  9022
frame:  9023
frame:  9024
frame:  9025
frame:  9026

frame:  9584
frame:  9585
frame:  9586
frame:  9587
frame:  9588
frame:  9589
frame:  9590
frame:  9591
frame:  9592
frame:  9593
frame:  9594
frame:  9595
frame:  9596
frame:  9597
frame:  9598
frame:  9599
frame:  9600
frame:  9601
frame:  9602
frame:  9603
frame:  9604
frame:  9605
frame:  9606
frame:  9607
frame:  9608
frame:  9609
frame:  9610
frame:  9611
frame:  9612
frame:  9613
frame:  9614
frame:  9615
frame:  9616
frame:  9617
frame:  9618
frame:  9619
frame:  9620
frame:  9621
frame:  9622
frame:  9623
frame:  9624
frame:  9625
frame:  9626
frame:  9627
frame:  9628
frame:  9629
frame:  9630
frame:  9631
frame:  9632
frame:  9633
frame:  9634
frame:  9635
frame:  9636
frame:  9637
frame:  9638
frame:  9639
frame:  9640
frame:  9641
frame:  9642
frame:  9643
frame:  9644
frame:  9645
frame:  9646
frame:  9647
frame:  9648
frame:  9649
frame:  9650
frame:  9651
frame:  9652
frame:  9653
frame:  9654
frame:  9655
frame:  9656
frame:  9657
frame:  9658
frame:  9659
frame:  9660

frame:  10199
frame:  10200
frame:  10201
frame:  10202
frame:  10203
frame:  10204
frame:  10205
frame:  10206
frame:  10207
frame:  10208
frame:  10209
frame:  10210
frame:  10211
frame:  10212
frame:  10213
frame:  10214
frame:  10215
frame:  10216
frame:  10217
frame:  10218
frame:  10219
frame:  10220
frame:  10221
frame:  10222
frame:  10223
frame:  10224
frame:  10225
frame:  10226
frame:  10227
frame:  10228
frame:  10229
frame:  10230
frame:  10231
frame:  10232
frame:  10233
frame:  10234
frame:  10235
frame:  10236
frame:  10237
frame:  10238
frame:  10239
frame:  10240
frame:  10241
frame:  10242
frame:  10243
frame:  10244
frame:  10245
frame:  10246
frame:  10247
frame:  10248
frame:  10249
frame:  10250
frame:  10251
frame:  10252
frame:  10253
frame:  10254
frame:  10255
frame:  10256
frame:  10257
frame:  10258
frame:  10259
frame:  10260
frame:  10261
frame:  10262
frame:  10263
frame:  10264
frame:  10265
frame:  10266
frame:  10267
frame:  10268
frame:  10269
frame:

frame:  10784
frame:  10785
frame:  10786
frame:  10787
frame:  10788
frame:  10789
frame:  10790
frame:  10791
frame:  10792
frame:  10793
frame:  10794
frame:  10795
frame:  10796
frame:  10797
frame:  10798
frame:  10799
frame:  10800
frame:  10801
frame:  10802
frame:  10803
frame:  10804
frame:  10805
frame:  10806
frame:  10807
frame:  10808
frame:  10809
frame:  10810
frame:  10811
frame:  10812
frame:  10813
frame:  10814
frame:  10815
frame:  10816
frame:  10817
frame:  10818
frame:  10819
frame:  10820
frame:  10821
frame:  10822
frame:  10823
frame:  10824
frame:  10825
frame:  10826
frame:  10827
frame:  10828
frame:  10829
frame:  10830
frame:  10831
frame:  10832
frame:  10833
frame:  10834
frame:  10835
frame:  10836
frame:  10837
frame:  10838
frame:  10839
frame:  10840
frame:  10841
frame:  10842
frame:  10843
frame:  10844
frame:  10845
frame:  10846
frame:  10847
frame:  10848
frame:  10849
frame:  10850
frame:  10851
frame:  10852
frame:  10853
frame:  10854
frame:

frame:  11373
frame:  11374
frame:  11375
frame:  11376
frame:  11377
frame:  11378
frame:  11379
frame:  11380
frame:  11381
frame:  11382
frame:  11383
frame:  11384
frame:  11385
frame:  11386
frame:  11387
frame:  11388
frame:  11389
frame:  11390
frame:  11391
frame:  11392
frame:  11393
frame:  11394
frame:  11395
frame:  11396
frame:  11397
frame:  11398
frame:  11399
frame:  11400
frame:  11401
frame:  11402
frame:  11403
frame:  11404
frame:  11405
frame:  11406
frame:  11407
frame:  11408
frame:  11409
frame:  11410
frame:  11411
frame:  11412
frame:  11413
frame:  11414
frame:  11415
frame:  11416
frame:  11417
frame:  11418
frame:  11419
frame:  11420
frame:  11421
frame:  11422
frame:  11423
frame:  11424
frame:  11425
frame:  11426
frame:  11427
frame:  11428
frame:  11429
frame:  11430
frame:  11431
frame:  11432
frame:  11433
frame:  11434
frame:  11435
frame:  11436
frame:  11437
frame:  11438
frame:  11439
frame:  11440
frame:  11441
frame:  11442
frame:  11443
frame: